In [4]:
import numpy as np
import pandas as pd
import Bio.SeqIO as SeqIO

In [2]:
data = pd.read_excel("/home/anwer/Desktop/PPI_prediction/Dataset_ppi/Denovo/AS1.xlsx")

In [3]:
data = data[['Protein A','Protein B']]

## 1. Data Preprocessing

### 1.1 Protein name to sequence

In [5]:
from tqdm import tqdm
dct_1 = {}
for rec in tqdm(SeqIO.parse("/home/anwer/Desktop/PPI_prediction/Dataset_ppi/Denovo/AS2.fasta","fasta")):
    dct_1[rec.id.split("|")[1]] = str(rec.seq)
for rec in tqdm(SeqIO.parse("/home/anwer/Desktop/PPI_prediction/Dataset_ppi/Denovo/AS3.fasta","fasta")):
    dct_1[rec.id.split("|")[1]] = str(rec.seq)    

445it [00:00, 50387.81it/s]
2340it [00:00, 57233.08it/s]


In [6]:
def convert_to_seq(seq_name):
    try:
        return dct_1[seq_name]
    except:
        return None

In [7]:
data["Protein_A_sequence"] = data['Protein A'].apply(convert_to_seq)
data["Protein_B_sequence"] = data['Protein B'].apply(convert_to_seq)
data.dropna(inplace = True)

In [42]:
data = data.apply(lambda x: x.str.strip())
data.to_csv("/home/anwer/Desktop/PPI_prediction/csv_files/Denovo_ppi_with_sequences.csv",index = False)

### 1.2 Exploratory Data Analysis

In [43]:
data = pd.read_csv('/home/anwer/Desktop/PPI_prediction/csv_files/Denovo_ppi_with_sequences.csv')

In [44]:
data.head()

,Protein A,Protein B,Protein_A_sequence,Protein_B_sequence
0,P29991,O00571,MNDQRKEAKNTPFNMLKRERNRVSTVQQLTKRFSLGMLQGRGPLKL...,MSHVAVENALGLDQQFAGLDLNSSDNQSGGSTASKGRYIPPHLRNR...
1,P29991,O00584,MNDQRKEAKNTPFNMLKRERNRVSTVQQLTKRFSLGMLQGRGPLKL...,MRPAALRGALLGCLCLALLCLGGADKRLRDNHEWKKLIMVQHWPET...
2,Q99IB8,O00584,MSTNPKPQRKTKRNTNRRPEDVKFPGGGQIVGGVYLLPRRGPRLGV...,MRPAALRGALLGCLCLALLCLGGADKRLRDNHEWKKLIMVQHWPET...
3,P03070,O00629,MDKVLNREESLQLMDLLGLERSAWGNIPLMRKAYLKKCKEFHPDKG...,MADNEKLDNQRLKNFKNKGRDLETMRRQRNEVVVELRKNKRDEHLL...
4,P49116,P10221,MTSPSPRIQIISTDSAVASPQRIQIVTDQQTGQKIQIVTAVDASGS...,MADRGLPSEAPVVTTSPAGPPSDGPMQRLLASLAGLRQPPTPTAET...


In [45]:
data = data.apply(lambda x: x.str.strip())

In [47]:
print("Number of positive pairs: ", len(data) )

Number of positive pairs:  5718


In [48]:
print("Number of unique Proteins: ", len(set(data['Protein A']).union(set(data['Protein B']))) )

Number of unique Proteins:  2781


In [49]:
print("Maximum length proteins: ", max(set(data['Protein_A_sequence']).union(set(data['Protein_B_sequence']))))
print('\n' )
print("Maximum length of Proteins sequences: ", len(max(set(data['Protein_A_sequence']).union(set(data['Protein_B_sequence'])))))

Maximum length proteins:  YITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKRGTRGGAGSMLQAPELPTKTRTCRRAAEQKASPPSLTPKLLRRQVTASPSSGLSHKKEATKGSASGMGTPATAEPAPPSNKVGLSKASSEEMRVRRHKHSSESPGRDKGRLAKLKPAPPPPPACTGKAGKPAQSPSQEAGEAGGPTKTKCTSLAMDAVNTDPTKAGPPGEGLRKPVPPSVPKPQSTAKPPGTPTSPVSTPSTAPAPSPLAGDQQPSSAAFIPLISTRVSLRKTRQPPERIASGTITKGVVLDSTEALCLAISRNSEQMASHSAVLEAGKNLYTFCVSYVDSIQQMRNKFAFREAINKLESNLRELQICPATASSGPAATQDFSKLLSSVKEISDIVRR


Maximum length of Proteins sequences:  746
